<p style = "font-weight:bold; font-size:200%">踏み台SSH</p>

# 前処理

## 接続情報

In [ ]:
gateway_host     = ''
gateway_user     = ''
gateway_password = ''
remote_host      = ''
remote_user      = ''
remote_password  = ''

## インポート

In [ ]:
import paramiko
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logging.basicConfig(
    format='[%(asctime)s] %(name)s %(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger.info('Done imports.')

## 関数

### コマンドを実行
標準出力と標準エラーを纏めて返す。

In [ ]:
def exec_command(client, command, sudo_passwd=''):
    logger.debug("[command] " + command)
    output = ""
    stdin, stdout, stderr = client.exec_command(command, get_pty=True)
    if stdout.channel.closed is False:
        stdin.write(sudo_passwd + "\n")
        stdin.flush()
    for line in stdout:
        output = output + line
    for line in stderr:
        output = output + line
    return output

# 接続

## SSH接続

In [ ]:
gateway = paramiko.SSHClient()
gateway.set_missing_host_key_policy(paramiko.AutoAddPolicy())
gateway.connect(
    hostname=gateway_host,
    username=gateway_user,
    password=gateway_password,
)
gateway

In [ ]:
sock = gateway.get_transport().open_channel(
    "direct-tcpip", (remote_host, 22), ('', 22)
)
remote = paramiko.SSHClient()
remote.set_missing_host_key_policy(paramiko.AutoAddPolicy())
remote.connect(
    hostname=remote_host,
    username=remote_user,
    password=remote_password,
    sock=sock
)
remote

# 処理実行

In [ ]:
print(exec_command(gateway,
    'hostname'
))

In [ ]:
print(exec_command(remote,
    'hostname'
))

# 切断

In [ ]:
remote.close()

In [ ]:
gateway.close()